In [1]:
from app.data_loader import load_documents

In [2]:
full_data = load_documents(
    "MSFT",
    "2022",
    "10-Q",
    False,
    6,
    "Q3"
)

Started for MSFT


In [3]:
full_data

(['', '', '', '', '', '', '', '', '', ''],
 [{'full_metadata': 'MSFT_202212_FINANCIAL_STATEMENTS_10-Q'},
  {'full_metadata': 'MSFT_202212_MANAGEMENT_DISCUSSION_10-Q'},
  {'full_metadata': 'MSFT_202212_MARKET_RISK_DISCLOSURES_10-Q'},
  {'full_metadata': 'MSFT_202212_CONTROLS_AND_PROCEDURES_10-Q'},
  {'full_metadata': 'MSFT_202212_LEGAL_PROCEEDINGS_10-Q'},
  {'full_metadata': 'MSFT_202212_RISK_FACTORS_10-Q'},
  {'full_metadata': 'MSFT_202212_USE_OF_PROCEEDS_10-Q'},
  {'full_metadata': 'MSFT_202212_DEFAULTS_10-Q'},
  {'full_metadata': 'MSFT_202212_MINE_SAFETY_10-Q'},
  {'full_metadata': 'MSFT_202212_OTHER_INFORMATION_10-Q'}])

In [17]:
from app.data_extraction import get_data

data = get_data(
    "NVDA",
    "2021",
    "10-K",
    # quarters="Q3"
)

Started for NVDA


In [18]:
data

defaultdict(list,
            {'NVDA': [{'year': '2022',
               'ticker': 'NVDA',
               'all_texts': {'BUSINESS': "NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields. Fueled by the sustained demand for exceptional 3D graphics and the scale of the gaming market, NVIDIA has leveraged its GPU architecture to create platforms for scientific computing, artificial intelligence, or AI, data science, autonomous vehicles, or AV, robotics, and augmented and virtual reality, or AR and VR. The GPU was initially used to simulate human imagination, enabling the virtual worlds of video games and films. Today, it also simulates human intelligence, enabling a deeper understanding of the physical world. Its parallel processing capabilities, supported by thousands of computing cores, are essential to running deep le

In [16]:
from app.utils import get_filing_urls_to_download

get_filing_urls_to_download(
    "10-K",
    "TSLA",
    1,
    "2022-01-01",
    "2023-10-31",
    include_amends=False,
    query=""
)

[FilingMetadata(accession_number='0000950170-23-001409', full_submission_url='https://www.sec.gov/Archives/edgar/data/0001318605/000095017023001409/0000950170-23-001409.txt', filing_details_url='https://www.sec.gov/Archives/edgar/data/0001318605/000095017023001409/tsla-20221231.htm', filing_details_filename='filing-details.html')]